To showcase the most important functions of the Aspen-Python-Interface provided by Zihao Wang (https://github.com/zwang1995/Aspen-Plus-Automation), a converged Aspen Plus file "Post_combustion_solvent_based_MEA.bkp" is opened and the initial performance of the process is returned. Then, the CO2 concentration of the FLUEGAS stream is changed, the simulation is executed and the change in performance is reviewed. Please note that in this example the CO2 removal efficiency is kept constant at 95 % by a design specification in Aspen Plus.

First, we import the library aspen_utils (developed by Mr. Wang) to facilitate the interaction of Python with Aspen Plus. Ensure that all the packages such as os, time and win32com.client required for the library to work are installed in your Python distribution. Then, it is recommended to get familiar with the structure of the library. It mainly consists of one class called Aspen_Plus_Interface(), in which several functions to interact with Aspen Plus are defined. In the actual code below, we will first generate an instance of this class and then use some of its functions for our purpose. 


In [10]:
from aspen_utils import *

# Create instance of the Aspen_Plus_Interface() class
Aspen_Plus = Aspen_Plus_Interface() 

# Use load_bkp function to open your Aspen Plus file.
Aspen_Plus.load_bkp(r"Aspen_File/Post_combustion_solvent_based_MEA.bkp", 0, 1)
        #Load a process via bkp file
        #:param bkp_file: location of the Aspen Plus file
        #:param visible_state: Aspen Plus user interface, 0 is invisible and 1 is visible
        #:param dialog_state: Aspen Plus dialogs, 0 is not to suppress and 1 is to suppress
print("\n")

Aspen Plus 40.0 OLE Services




After running the cell above you should get a notification with something like "Aspen Plus 40.0 OLE Services". Since the file we opened was already run and results should be available, we want to take a look at the initial state. To extract a value from Aspen Plus and assign it to a variable in Python, we first need to figure out, how Aspen Plus declares this variable in the simulation. You can do that within Aspen Plus by the Variable Explorer. 

![Alt-Text](Aspen_Plus_Variable_Explorer.png)

For example, if we want to extract the value of the reboiler duty required for the stripper, we find the file path in the Variable Explorer like this: 

![Alt-Text](Aspen_Plus_Variable_Explorer2.png)

With this approach, most of the process parameters can be extracted from Aspen Plus with the command Aspen_Plus.Application.Tree.FindNode(r"Filepath_Variable_Explorer").Value. Lets test it:

In [11]:
###################################################################################################################
# 1. Return performance of initial process parameter configuration
###################################################################################################################

print("1. Return performance of initial process parameter configuration")
print("------------------------------------------------------------------------------------------------------------\n")

# Get reboiler duty required for the stripper in kW
STRIPPER_ReboilerDuty = Aspen_Plus.Application.Tree.FindNode(r"\Data\Blocks\REBOILER\Output\QCALC").Value * 1e-03
print("Required reboiler duty in Stripper: ", round(STRIPPER_ReboilerDuty,1), " kW")

# Get the amount of CO2 captured by the process in kg/h (see Aspen Plus File - stream CO2OUT)
CO2OUT_CO2CaptureRate = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\CO2-OUT\Output\STR_MAIN\MASSFLOW\MIXED\CO2").Value *3600
print("Captured amount of CO\u2082: ", round(CO2OUT_CO2CaptureRate,1), " kg/h")

# Calculate performance of initial process configuration
STRIPPER_SpecReboilerDuty = round((STRIPPER_ReboilerDuty / (CO2OUT_CO2CaptureRate / 3600)), 5)
print("Performance measured by reboiler duty per ton CO\u2082 captured: ", round(STRIPPER_SpecReboilerDuty,1), " MJ/ ton CO\u2082\n")

1. Return performance of initial process parameter configuration
------------------------------------------------------------------------------------------------------------

Required reboiler duty in Stripper:  912.1  kW
Captured amount of CO₂:  864.0  kg/h
Performance measured by reboiler duty per ton CO₂ captured:  3800.5  MJ/ ton CO₂



To assess the performance of the initial process configuration, the reboiler duty per ton CO2 captured is used. Typically, this value is in a range of 3.2 - 4.0 MJ/ton CO2 with Monoethanolamine (MEA) as a solvent.

Lets see how an increase of the CO2 content in the flue gas entering the absorber impacts this performance metric. First, we extract the initial CO2 and N2 content from Aspen Plus with the same approach as above. Similarly, we can increase the CO2 concentration by 0.1 vol%. If we increase the percentage of one component (CO2), we must reduce the percentage of another component (N2) accordingly. Additionally, we fixed the CO2 removal efficieny in the beginning to 95% with a design specification. Thus, we also have to adapt it to ensure a constant CO2 removal efficiency for all runs.

In [12]:
###################################################################################################################
# 2. Modify process parameters
###################################################################################################################

print("2. Modify process parameters")
print("------------------------------------------------------------------------------------------------------------\n")

# Get initial CO2 and N2 concentration in FLUEGAS stream
Fluegas_CO2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\CO2").Value * 100
print("Initial CO\u2082 concentration in FLUEGAS stream: ", round(Fluegas_CO2conc,1), " vol%")

Fluegas_N2conc = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\N2").Value * 100
print("Initial N\u2082 concentration in FLUEGAS stream: ", round(Fluegas_N2conc,1), " vol%\n")

# Increase CO2 concentration in the FLUEGAS stream by 0.1 vol% and adjust N2 concentration accordingly 
Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\CO2").Value = (Fluegas_CO2conc + 0.1)/100 
Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Input\FLOW\MIXED\N2").Value = (Fluegas_N2conc - 0.1)/100

# Since the CO2 concentration in the FLUEGAS stream is changed, the design specification in Aspen Plus for keeping the 
# CO2 removal efficiency constant also needs to be adjusted:

# 1. Get the new value of CO2, which needs to be captured (95 % CO2 removal efficiency)
DS_TargetValue = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\FLUEGAS\Output\MASSFLOW\MIXED\CO2").Value * 0.95 * 3600

# 2. Adjust the Design Specification in Aspen Plus accordingly
Aspen_Plus.Application.Tree.FindNode(r"\Data\Flowsheeting Options\Design-Spec\DS-1\Input\EXPR2").Value = DS_TargetValue



2. Modify process parameters
------------------------------------------------------------------------------------------------------------

Initial CO₂ concentration in FLUEGAS stream:  10.1  vol%
Initial N₂ concentration in FLUEGAS stream:  74.9  vol%

3. Simulation run and trouble monitoring
------------------------------------------------------------------------------------------------------------

Simulation running. It can take a few seconds. Please wait...
No errors encountered. Ready to extract simulation results.

---- Results for adjusted CO2 concentration: ----
Required reboiler duty in Stripper:  909.7  kW
Captured amount of CO₂:  864.3  kg/h
Performance measured by reboiler duty per ton CO₂ captured:  3788.7  MJ/ ton CO₂



In [ ]:
###################################################################################################################
# 3. Simulation run and trouble monitoring
###################################################################################################################

print("3. Simulation run and trouble monitoring")
print("------------------------------------------------------------------------------------------------------------\n")
print("Simulation running. It can take a few seconds. Please wait...")

# Run the process simulation
Aspen_Plus.run_simulation()
Aspen_Plus.check_run_completion()

# Trouble monitoring
# Aspen gives these numbers as an output regarding the success of a simulation run. (They are based on hexadecimal codes).
sim_success = 9345  # Hexadecimal: 0x00002481
sim_warning = 9348  # Hexadecimal: 0x00002484
sim_error = 9376    # Hexadecimal: 0x000024A0

run_status = Aspen_Plus.Application.Tree.FindNode(r"\Data").AttributeValue(12)

if run_status == sim_success:
    print("No errors encountered. Ready to extract simulation results.\n")

    print("---- Results for adjusted CO2 concentration: ----")
    
    # Extracting results as described in 1.
    New_STRIPPER_ReboilerDuty = Aspen_Plus.Application.Tree.FindNode(r"\Data\Blocks\REBOILER\Output\QCALC").Value * 1e-03
    print("Required reboiler duty in Stripper: ", round(New_STRIPPER_ReboilerDuty,1), " kW")
    
    New_CO2OUT_CO2CaptureRate = Aspen_Plus.Application.Tree.FindNode(r"\Data\Streams\CO2-OUT\Output\STR_MAIN\MASSFLOW\MIXED\CO2").Value *3600
    print("Captured amount of CO\u2082: ", round(New_CO2OUT_CO2CaptureRate,1), " kg/h")
    
    New_STRIPPER_SpecReboilerDuty = round((New_STRIPPER_ReboilerDuty / (New_CO2OUT_CO2CaptureRate / 3600)), 5)
    print("Performance measured by reboiler duty per ton CO\u2082 captured: ", round(New_STRIPPER_SpecReboilerDuty,1), " MJ/ ton CO\u2082\n")

elif run_status == sim_warning:
    print("Warnings encountered.")
else:
    print("Errors encountered.")

# PLEASE NOTE that in order to obtain 100% correct results, the mass balance of the LEANMEA and LEANMEAC streams 
# must be matched to each other. As this is an introductory example, this is not carried out here.